### hammurabiX synchrotron foreground precision check

In [ ]:
import matplotlib
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import FormatStrFormatter
import hampyx as ham
import scipy.special as sp
matplotlib.use('Agg')

%matplotlib inline

# CGS units
pi = 3.14159265358979
kpc = 3.0856775806e+21
eV = 1.60217733e-12
GeV = 1.e+9*eV
GHz = 1.e9
mG = 1.e-6
q = 4.8032068e-10
mc2 = 0.51099907e-3*GeV
cc = 2.99792458e+10
mc = mc2/cc
kB = 1.380622e-16
h = 6.626075540e-27


def t_conv(_t, _freq):
    """
    convert T_br into T_cmb
    :param _t: birghtness temperature 
    :param _freq: observing frequency
    :return: cmb blackbody temperature (fluctuation)
    """
    p = (h*_freq*GHz)/(kB*2.725)
    return _t*(np.exp(p)-1.)**2/(p**2*np.exp(p))


def j_tot(_theta):
    """
    calculate total emissivity
    :param theta: parameter set
    :return: total emissivity in mK_cmb
    """
    b, r, l0, alpha, je, freq = _theta
    # CRE normalization
    gamma10 = 10.*GeV/mc2+1.0
    beta10 = np.sqrt(1.-1./gamma10)
    rslt = je*4.*pi*mc*(gamma10**alpha)/(1.e+4*GeV*beta10)
    # following Ribiki-Lightman eq(6.36)
    rslt *= np.sqrt(3)*(q**3)*b*mG/(mc2*(alpha+1))
    rslt *= sp.gamma(0.25*alpha+19./12.)*sp.gamma(0.25*alpha-1./12.)
    rslt *= (mc*2.*pi*freq*GHz/(3.*q*b*mG))**(0.5-0.5*alpha)
    # convert into mK_br
    rslt *= 1.e+3*cc*cc/(2.*kB*freq*GHz*freq*GHz)
    # convert into mK_cmb
    rslt = t_conv(rslt, freq)
    return rslt/(4.*pi)


def j_pol(_theta):
    """
    calculate polarized emissivity
    :param theta: parameter set
    :return: total emissivity in mK_cmb
    """
    b, r, l0, alpha, je, freq = _theta
    # CRE normalization
    gamma10 = 10.*GeV/mc2+1.0
    beta10 = np.sqrt(1.-1./gamma10)
    rslt = je*4.*pi*mc*(gamma10**alpha)/(1.e+4*GeV*beta10)
    # following Ribiki-Lightman eq(6.36)
    rslt *= np.sqrt(3)*(q**3)*b*mG/(4.*mc2)
    rslt *= sp.gamma(0.25*alpha+7./12.)*sp.gamma(0.25*alpha-1./12.)
    rslt *= (mc*2.*pi*freq*GHz/(3.*q*b*mG))**(0.5-0.5*alpha)
    # convert into mK_br
    rslt *= 1.e+3*cc*cc/(2.*kB*freq*GHz*freq*GHz)
    # convert into mK_cmb
    rslt = t_conv(rslt, freq)
    return rslt/(4.*pi)


def i_th(_theta, _lon, _lat):
    """
    calculate synchrotron foreground total intensity per unit radius (in mK_cmb/cm)
    :param _theta:
    :param _lon:
    :param _lat:
    :return:
    """
    tmp = _theta
    lon = (_lon - tmp[2])*np.pi/180.
    lat = _lat*np.pi/180.
    # b_perp
    tmp[0] *= np.sqrt(1+tmp[1]**2 - (np.cos(lat)*np.cos(lon) + tmp[1]*np.sin(lat))**2)
    return j_tot(tmp)


def pi_th(_theta, _lon, _lat):
    """
    calculate synchrotron foreground polarized intensity per unit radius (in mK_cmb/cm)
    without Faraday rotation correction
    :param theta:
    :return:
    """
    tmp = _theta
    lon = (_lon - tmp[2])*np.pi/180.
    lat = _lat*np.pi/180.
    # b_perp
    tmp[0] *= np.sqrt(1+tmp[1]**2 - (np.cos(lat)*np.cos(lon) + tmp[1]*np.sin(lat))**2)
    return j_pol(tmp)


def q_th(_theta, _lon, _lat):
    """
    synchrotron Stokes Q per unit radius (in mK_cmb/cm)
    without Faraday rotation correction
    :param theta:
    :param lon:
    :param lat:
    :return:
    """
    tmp = _theta
    lon = (_lon - tmp[2])*np.pi/180.
    lat = _lat*np.pi/180.
    # b_perp
    tmp[0] *= np.sqrt(1+tmp[1]**2 - (np.cos(lat)*np.cos(lon) + tmp[1]*np.sin(lat))**2)
    # cos(2X)
    numerator = np.sin(lon)**2 - (tmp[1]*np.cos(lat)-np.sin(lat)*np.cos(lon))**2
    denominator = np.sin(lon)**2 + (tmp[1]*np.cos(lat)-np.sin(lat)*np.cos(lon))**2
    return j_pol(tmp)*numerator/denominator


def u_th(_theta, _lon, _lat):
    """
    synchrotron Stokes U per unit radius (in mK_cmb/cm)
    without Faraday rotation correction
    :param theta:
    :param lon:
    :param lat:
    :return:
    """
    tmp = _theta
    lon = (_lon - tmp[2])*np.pi/180.
    lat = _lat*np.pi/180.
    # b_perp
    tmp[0] *= np.sqrt(1+tmp[1]**2 - (np.cos(lat)*np.cos(lon) + tmp[1]*np.sin(lat))**2)
    # sin(2X)
    numerator = 2.*np.sin(lon)*(tmp[1]*np.cos(lat)-np.sin(lat)*np.cos(lon))
    denominator = np.sin(lon)**2 + (tmp[1]*np.cos(lat)-np.sin(lat)*np.cos(lon))**2
    return j_pol(tmp)*numerator/denominator


def fd_th(_theta, _lon, _lat):
    """
    calculate Faraday depth per unit radius (in rad/cm3)
    :param theta:
    :param lon:
    :param lat:
    :return:
    """
    tmp = _theta
    lon = (_lon - tmp[2])*np.pi/180.
    lat = _lat*np.pi/180.
    # b_parallel
    tmp[0] *= (np.cos(lat)*np.cos(lon) + tmp[1]*np.sin(lat))
    return tmp[-1]*tmp[0]*mG*(-q**3/(2*pi*mc2**2))


def precision_dist(_freq, _res):
    """
    precision
    :param _res:
    :return:
    """
    # observable controllers
    nside = 32
    shell = 2
    res = _res  # affecting precision
    radius = 4.0  # total LoS radius

    # field controllers
    b0 = 6.0  # muG
    r = 0
    l0 = 0.0
    alpha = 3
    je = 0.25
    freq = _freq  # affecting precision
    ne = 0.01  # pccm
    E0 = 10  # by definition

    # call hammurabiX wrapper
    obj = ham.Hampyx(exe_path='/home/lab/hamx/build/hamx')
    # assuming the xml file is not prepared
    obj.del_par(['observable', 'sync'], 'all')
    obj.add_par(['observable'], 'sync', {'cue': str(1),
                                         'freq': str(freq),
                                         'filename': 'dumy',
                                         'nside': str(nside)})
    obj.mod_par(['observable', 'dm'], {'cue': str(1), 'nside': str(nside)})
    obj.mod_par(['observable', 'faraday'], {'cue': str(1), 'nside': str(nside)})
    # mute all field output/input
    obj.del_par(['fieldio', 'breg'], 'all')
    obj.del_par(['fieldio', 'brnd'], 'all')
    obj.del_par(['fieldio', 'fereg'], 'all')
    obj.del_par(['fieldio', 'fernd'], 'all')
    obj.del_par(['fieldio', 'cre'], 'all')
    # calibrate simulation box
    obj.mod_par(['grid', 'shell', 'layer'], {'type': 'auto'})
    #obj.mod_par(['grid', 'shell', 'layer'], {'type': 'manual'})
    obj.mod_par(['grid', 'shell', 'layer', 'auto', 'shell_num'], {'value': str(shell)})
    obj.mod_par(['grid', 'shell', 'layer', 'auto', 'nside_sim'], {'value': str(nside)})
    
    obj.mod_par(['grid', 'shell', 'ec_r_max'], {'value': str(radius)})
    obj.mod_par(['grid', 'shell', 'gc_r_max'], {'value': str(radius+9.)})
    obj.mod_par(['grid', 'shell', 'gc_z_max'], {'value': str(radius+1.)})
    obj.mod_par(['grid', 'shell', 'ec_r_res'], {'value': str(res)})
    # fix GMF
    obj.mod_par(['magneticfield', 'regular'], {'cue': str(1), 'type': 'unif'})
    obj.mod_par(['magneticfield', 'regular', 'unif', 'b0'], {'value': str(b0)})
    obj.mod_par(['magneticfield', 'regular', 'unif', 'l0'], {'value': str(l0)})
    obj.mod_par(['magneticfield', 'regular', 'unif', 'r'], {'value': str(r)})
    obj.mod_par(['magneticfield', 'random'], {'cue': str(0)})
    # fix FE
    obj.mod_par(['freeelectron', 'regular'], {'cue': str(1), 'type': 'unif'})
    obj.mod_par(['freeelectron', 'regular', 'unif', 'n0'], {'value': str(ne)})
    obj.mod_par(['freeelectron', 'regular', 'unif', 'r0'], {'value': str(radius)})
    obj.mod_par(['freeelectron', 'random'], {'cue': str(0)})
    # fix CRE
    obj.mod_par(['cre'], {'type': 'unif'})
    obj.mod_par(['cre', 'unif', 'alpha'], {'value': str(alpha)})
    obj.mod_par(['cre', 'unif', 'E0'], {'value': str(10.0)})
    obj.mod_par(['cre', 'unif', 'j0'], {'value': str(je)})
    obj.mod_par(['cre', 'unif', 'r0'], {'value': str(radius)})
    # call hammurabi executable
    obj(True)
    # (in mK_cmb)
    qsim = obj.sim_map[('sync', str(freq), str(nside), 'Q')]*1.e+3
    usim = obj.sim_map[('sync', str(freq), str(nside), 'U')]*1.e+3
    isim = obj.sim_map[('sync', str(freq), str(nside), 'I')]*1.e+3
    fsim = obj.sim_map[('fd', 'nan', str(nside), 'nan')]

    ith = np.zeros_like(isim)
    pith = np.zeros_like(isim)
    qth = np.zeros_like(qsim)
    uth = np.zeros_like(usim)
    fth = np.zeros_like(fsim)
    for i in range(0, np.size(qth)):
        l, b = hp.pix2ang(nside, i, lonlat=True)
        ith[i] = i_th([b0, r, l0, alpha, je, freq], l, b)*radius*kpc
        pith[i] = pi_th([b0, r, l0, alpha, je, freq], l, b)*radius*kpc
        qth[i] = q_th([b0, r, l0, alpha, je, freq], l, b)*radius*kpc
        uth[i] = u_th([b0, r, l0, alpha, je, freq], l, b)*radius*kpc
        fth[i] = fd_th([b0, r, l0, ne], l, b)*radius*kpc*1.e+4
    
    x = 2.*fth*(0.09/freq**2)
    #print (0.09/freq**2)
    #print ('radius ', radius*kpc)
    qthc = ( qth*np.sin(x) - uth*(1-np.cos(x)) )/x
    uthc = ( uth*np.sin(x) + qth*(1-np.cos(x)) )/x
    
    #qthc = ( qth*np.cos(x) - uth*np.sin(x) )
    #uthc = ( uth*np.cos(x) + qth*np.sin(x) )
    
    # correction to pol. intensity from Faraday rotation
    #picorrect = 2.*abs(np.sin(0.5*x)/x)
    #pith = pith * picorrect
    
    pisim = np.sqrt(qsim**2+usim**2)

    #hp.mollview(fth,norm='hist')
    #hp.mollview(fsim,norm='hist')
    #hp.mollview((fsim-fth)/fth)
    
    #hp.mollview(ith,norm='hist',cmap='coolwarm')
    #hp.mollview(isim,norm='hist',cmap='coolwarm')
    #hp.mollview((isim-ith),norm='hist',cmap='coolwarm')
    #matplotlib.rcParams.update({'font.size': 15})
    #hp.mollview((isim-ith)/ith,format='%.2g',cmap='coolwarm',title='scalar, 1 shell')
    #plt.savefig('err_spin0_1shell.pdf')
    
    #hp.mollview(qthc,norm='hist',cmap='coolwarm')
    #hp.mollview(qsim,norm='hist',cmap='coolwarm')
    #hp.mollview((qsim-qthc)/qthc,cmap='coolwarm')
    
    #hp.mollview(uthc,norm='hist',cmap='coolwarm')
    #hp.mollview(usim,norm='hist',cmap='coolwarm')
    #hp.mollview((usim-uthc)/uthc,min=-0.1,max=0.1,cmap='coolwarm')
    
    #hp.mollview(pith,norm='hist',cmap='coolwarm')
    #hp.mollview(pisim,norm='hist',cmap='coolwarm')
    #hp.mollview((pisim-pith),norm='hist',cmap='coolwarm')
    #hp.mollview((pisim-pith)/pith,cmap='coolwarm')
    
    fig = plt.figure(figsize=(9,9))
    gs = matplotlib.gridspec.GridSpec(10, 10)
    ax1 = fig.add_subplot(gs[0:6, 0:8])
    
    hp.mollview(qsim,norm='hist',cmap='coolwarm',sub=(1,2,1),hold=True,title=None,cbar=0)
    
    ax2 = fig.add_subplot(gs[4:10, 1:9])
    
    matplotlib.rcParams.update({'font.size': 15})
    hp.mollview((qsim-qthc)/qthc,min=-0.1,max=0.1,cmap='coolwarm',sub=(1,2,2),hold=True,title=None,cbar=1)
    
    ax3 = fig.add_subplot(gs[3:5,7:10])
    ax3.hist((qsim-qthc)/qthc,50,range=(-0.01,0.01))
    ax3.set_yticklabels([])
    ax3.tick_params(axis='both', which='major', labelsize='20', color='steelblue')
    
    #plt.savefig('err_spin2_2shell.pdf')

In [ ]:
precision_dist(2.4,0.001)

end